In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from pyomo.environ import *

# Module relatif à Gurobi
from gurobipy import *

# First Basic model

In [2]:
# -- Initialisation du modèle --
# m : Model
m=Model("assign_SR")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-30


In [3]:
distances = pd.read_csv("opsto-ass1-dij2.csv", header=None)
Dij = 10*distances.to_numpy()
print(Dij)
print(Dij.shape)

[[nan 18. nan nan nan nan 10. nan nan nan nan nan 54.]
 [18. nan 17. nan 70. nan 20. 23. nan nan nan 30. nan]
 [nan 17. nan 20. nan nan nan nan nan 65. nan nan nan]
 [nan nan 20. nan 10. 20. nan nan nan nan nan nan nan]
 [nan 70. nan 10. nan nan nan 50. nan 10. 15. nan nan]
 [nan nan nan 20. nan nan nan nan nan nan 21. nan nan]
 [10. 20. nan nan nan nan nan nan nan nan nan 20. nan]
 [nan 23. nan nan 50. nan nan nan 20. nan nan nan  7.]
 [nan nan nan nan nan nan nan 20. nan 11. nan nan nan]
 [nan nan 65. nan 10. nan nan nan 11. nan 10. nan nan]
 [nan nan nan nan 15. 21. nan nan nan 10. nan nan nan]
 [nan 30. nan nan nan nan 20. nan nan nan nan nan 25.]
 [54. nan nan nan nan nan nan  7. nan nan nan 25. nan]]
(13, 13)


In [4]:
# Generating the Vij matrix:
import math

for i in range(Dij.shape[0]):
    for j in range(Dij.shape[1]):
        if i == j:
            Dij[i,j]= 0
        if math.isnan(Dij[i,j]):
            Dij[i,j]= 0

In [5]:
K = np.arange(3)  #subnetworks
V = np.arange(13) # nodes
c = 25 #price of backbone
card = 2 #cardinality if the system

In [6]:
x = m.addMVar(shape = (13,3), name='x', vtype=GRB.BINARY)
print(x.shape)
w = m.addMVar(shape = (13,13,3,3), name='w',lb=0, ub=1, vtype=GRB.CONTINUOUS,)

(13, 3)


In [7]:
# Adding some constraints
for i in V:
    m.addConstr(quicksum(x[i,:])==1)

for h in K:
    m.addConstr(quicksum(x[:,h])>= card)

In [8]:
for i in V:
    for j in V:
        for h in K:
            for k in K:
                if h != k: 
                    m.addConstr(w[i,j,h,k]>=(x[i,h]+x[j,k]-1))

In [9]:
# minimize cost

objective = 0.5* np.sum(c*Dij[i,j]*w[i,j,h,k] for h in K for k in K if h!=k for i in V for j in V if Dij[i,j]>0)

m.setObjective(objective, GRB.MINIMIZE)

m.params.outputflag = 0
m.update()
m.optimize()

if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    pass
elif m.status == GRB.UNBOUNDED:
    pass
else:
    print('the minimum cost is: ',m.ObjVal)

the minimum cost is:  2325.0


/var/folders/t4/9fy1pq9d6tjchq8lc65_mnn80000gn/T/ipykernel_41020/368024877.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  objective = 0.5* np.sum(c*Dij[i,j]*w[i,j,h,k] for h in K for k in K if h!=k for i in V for j in V if Dij[i,j]>0)


In [10]:
for h in K:
    print('subnetwork ',h+1,':')
    for i in V:
        if x.x[i,h]==1:
            print(i+1)


subnetwork  1 :
3
4
10
subnetwork  2 :
6
11
subnetwork  3 :
1
2
5
7
8
9
12
13
